3.Read in the file near_IR.fits. This file lists additional near-IR photometry for the galaxies in the file optical.txt, but the sources are not listed in the same order.

a. Append the dictionary you created earlier so that is has the columns:
name, ra ,dec, redshift, flux_u, u_err, flux_g, g_err, flux_r, r_err, flux_i, i_err, flux_z, z_err, flux_Y, Y_err, flux_J, J_err, flux_H, H_err, flux_K, K_err

In [1]:
import astropy as astro
import numpy as np
from astropy.table import Table
from astropy.io import fits
print("packages imported")

packages imported


In [4]:
# Reading optical data from previous session 
t_opt = Table.read('data/optical_tag.txt', format = 'ascii.commented_header')
print("read optical_tag.txt")

read optical_tag.txt


In [6]:
# Read file near_IR.fits
t_IR = Table.read('data/near_ir.fits')
t_IR # Print table as output

name,Y,Y_err,J,J_err,H,H_err,K,K_err
str8,float64,float64,float64,float64,float64,float64,float64,float64
3324m091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0136p000,0.0,0.0,0.0653014,0.00358461,0.119733,0.00714473,0.135493,0.00560361
0105m016,0.0567706,0.00339172,0.0482037,0.00438435,0.0874625,0.00659749,0.107061,0.00629918
0317p015,0.0776212,0.0037513,0.0693173,0.00538721,0.103825,0.00689423,0.126122,0.00644539
3221p000,0.18188,0.00455901,0.26426,0.00463779,0.365529,0.00819638,0.428345,0.00676489
0211p000,0.0701774,0.00396726,0.0988794,0.00583389,0.180533,0.00748821,0.354007,0.00717859
0242p000,0.0569324,0.00409066,0.0347468,0.00520014,0.0582967,0.00707859,0.0574337,0.00636905
0287m016,0.0496609,0.00364349,0.0651956,0.00463052,0.0914773,0.0058773,0.140448,0.00575708
0121p000,0.0402738,0.00288043,0.0705959,0.00390204,0.0946611,0.00643475,0.13383,0.0057457


In [7]:
colJ  = Table.Column(name='J', data = range(0,9), dtype=float)
colJe = Table.Column(name='J_err', data = range(0,9), dtype=float)
colY  = Table.Column(name='Y', data = range(0,9), dtype=float)
colYe = Table.Column(name='Y_err', data = range(0,9), dtype=float)
colH  = Table.Column(name='H', data = range(0,9), dtype=float)
colHe = Table.Column(name='H_err', data = range(0,9), dtype=float)
colK  = Table.Column(name='K', data = range(0,9), dtype=float)
colKe = Table.Column(name='K_err', data = range(0,9), dtype=float)
print("columns created")

t_opt.add_column(colJ) 
t_opt.add_column(colJe) 
t_opt.add_column(colY) 
t_opt.add_column(colYe) 
t_opt.add_column(colH) 
t_opt.add_column(colHe)
t_opt.add_column(colK) 
t_opt.add_column(colKe) 
print("columns added")

columns created
columns added


In [8]:
for i in np.arange(9):
    ind = np.where(t_IR['name'] == t_opt['id'][i])
    t_opt['J'][i] =  t_IR['J'][ind[0][0]]
    t_opt['J_err'][i] =  t_IR['J_err'][ind[0][0]]
    t_opt['Y'][i] =  t_IR['Y'][ind[0][0]]
    t_opt['Y_err'][i] =  t_IR['Y_err'][ind[0][0]]
    t_opt['H'][i] =  t_IR['H'][ind[0][0]]
    t_opt['H_err'][i] =  t_IR['H_err'][ind[0][0]]
    t_opt['K'][i] =  t_IR['K'][ind[0][0]]
    t_opt['K_err'][i] =  t_IR['K_err'][ind[0][0]]
print("data loaded")    

data loaded


b. Find the galaxy which has a u-band flux greater than 0.01 mJy but no detection in the Y-band. (print on screen)

In [9]:
idx  = np.where( (t_opt['u'] > 0.01) & (t_opt['Y'] == 0.0) )        
# you can also use idx = (t_opt['u'] > 0.01) & (t_opt['Y'] == 0.0)
print t_opt['id'][idx[0][0]]


3324m091


4.Read in the W1 images (the file names tell you which galaxy the image shows).

a. Find the maximum pixel value (x_cen, y_cen).

b. Find the sum of pixel values in a 11 pixel times 11 pixel box,
centered on (x_cen, y_cen) and multiply that sum by 1.93e-3. Add to the dictionary with key name W1. Take sqrt(W1) as W1_err and add to the dictionary.

In [10]:
#Crete the array for storing the fluxes
W1     = np.zeros([9])
W1_err = np.zeros([9])

#Loop to read fits, and store the fluxes in central region.
for i,val in enumerate(t_opt['id']):
    im = fits.getdata('ir_images/w1/'+val+'_ac51-w1.fits')
    max_loc = np.where(im ==  np.amax(im)) #4a
    region = im[max_loc[0][0]-11:max_loc[0][0]+11,max_loc[1][0]-11:max_loc[1][0]+11]
    tot = np.sum(region) # 4b
    print tot
    W1[i] = tot*1.93e-3    
    W1_err[i] = np.sqrt(tot*1.93e-3)
print W1
colW1  = Table.Column(name='W1', data = W1, dtype=float)
colW1e = Table.Column(name='W1_err', data = W1_err, dtype=float)
t_opt.add_column(colW1) 
t_opt.add_column(colW1e) 

IOError: [Errno 2] No such file or directory: 'ir_images/w1/3221p000_ac51-w1.fits'

5.Repeat 4a and 4b for the W2, W3 and W4 images and multiply the W2 values by 2.7e-3, the W3 values by 2.9e-3 and the W3 values by 5.2e-3, respectively. This converts the flux units to mJy. Add the values to the dictionary. Note that the W images are not
in the same galaxy order as your dictionary probably is. Make sure
to append correctly.

In [7]:
W2, W2_err, W3, W3_err, W4, W4_err = np.zeros([9]), np.zeros([9]), np.zeros([9]),np.zeros([9]),np.zeros([9]),np.zeros([9])
for i,val in enumerate(t_opt['id']):
    im2 = fits.getdata('IR_images/W2/'+val+'_ac51-w2.fits')
    im3 = fits.getdata('IR_images/W3/'+val+'_ac51-w3.fits')
    im4 = fits.getdata('IR_images/W4/'+val+'_ac51-w4.fits')

    max_loc2 = np.where(im2 ==  np.amax(im2)) #4a
    max_loc3 = np.where(im3 ==  np.amax(im3)) #4a
    max_loc4 = np.where(im4 ==  np.amax(im4)) #4a

    region2 = im2[max_loc2[0][0]-11:max_loc2[0][0]+11,max_loc2[1][0]-11:max_loc2[1][0]+11]
    region3 = im3[max_loc3[0][0]-11:max_loc3[0][0]+11,max_loc3[1][0]-11:max_loc3[1][0]+11]
    region4 = im4[max_loc4[0][0]-11:max_loc4[0][0]+11,max_loc4[1][0]-11:max_loc4[1][0]+11]

    tot2 = np.sum(region2) # 4b
    tot3 = np.sum(region3) # 4b
    tot4 = np.sum(region4) # 4b

    W2[i] = tot2*2.7e-3    
    W2_err[i] = np.sqrt(tot2*2.7e-3)
    W3[i] = tot3*2.9e-3    
    W3_err[i] = np.sqrt(tot3*2.9e-3)
    W4[i] = tot4*5.2e-3    
    W4_err[i] = np.sqrt(tot4*5.2e-3)

colW2  = Table.Column(name='W2', data = W2, dtype=float)
colW2e = Table.Column(name='W2_err', data = W2_err, dtype=float)
colW3  = Table.Column(name='W3', data = W3, dtype=float)
colW3e = Table.Column(name='W3_err', data = W3_err, dtype=float)
colW4  = Table.Column(name='W4', data = W4, dtype=float)
colW4e = Table.Column(name='W4_err', data = W4_err, dtype=float)

t_opt.add_column(colW2) 
t_opt.add_column(colW2e) 
t_opt.add_column(colW3) 
t_opt.add_column(colW3e) 
t_opt.add_column(colW4) 
t_opt.add_column(colW4e) 

6.Convert the dictionary to a Table (be careful with the order of the
keys in the dictionary) and write the Table to a text file.

In [8]:
t_opt # This is similar to the command t_opt.show_in_browser()

id,ra,dec,redshift,u,u_err,g,g_err,r,r_err,i,i_err,z,z_err,J,J_err,Y,Y_err,H,H_err,K,K_err,W1,W1_err,W2,W2_err,W3,W3_err,W4,W4_err
string64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
3221p000,321.724271547,0.598066621013,0.2349,0.0351163,0.00240797,0.104388,0.00128226,0.308084,0.00223131,0.459592,0.00357667,0.56537,0.0106389,0.26426,0.00463779,0.18188,0.00455901,0.365529,0.00819638,0.428345,0.00676489,4.21749009277,2.05365286569,13.3137777832,3.64880498015,900.40740625,30.0067893359,509.58090625,22.5738987827
3324m091,333.074690104,-9.73551338064,0.1805,0.0523519,0.00319408,0.181128,0.00170665,0.440053,0.00268839,0.629704,0.00392663,0.716125,0.0124855,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1143.82651875,33.820504413,669.75715625,25.8796668497
0105m016,10.9171472474,-0.863919513687,0.5849,0.00492203,0.000865933,0.00828142,0.000415467,0.0185295,0.000625145,0.0382226,0.000929956,0.0369601,0.00309237,0.0482037,0.00438435,0.0567706,0.00339172,0.0874625,0.00659749,0.107061,0.00629918,3.34666170044,1.82938834052,16.1293003418,4.01613002053,1179.98081875,34.3508488796,695.1436375,26.3655767527
0121p000,12.540912127,-0.650179379828,0.7276,0.016488,0.00104897,0.0227888,0.000503114,0.03806,0.000723773,0.0421127,0.000984811,0.116346,0.00367073,0.0705959,0.00390204,0.0402738,0.00288043,0.0946611,0.00643475,0.13383,0.0057457,2.60564654053,1.61420151794,24.5239998047,4.95217122126,1276.5151125,35.7283516622,741.80234375,27.2360486075
0136p000,14.0905335324,0.543261294954,0.484,-0.00403936,0.000647207,0.00939846,0.000383069,0.0190022,0.000567523,0.0487278,0.000977344,0.0499101,0.00289048,0.0653014,0.00358461,0.0,0.0,0.119733,0.00714473,0.135493,0.00560361,4.35800643799,2.08758387568,16.9355285156,4.11527988302,1265.747775,35.5773491846,731.420625,27.0447892393
0211p000,20.9227566654,0.743306927928,0.399,0.00615506,0.00123989,0.0170807,0.00064249,0.0327824,0.000985925,0.0819394,0.00169918,0.0937952,0.00566314,0.0988794,0.00583389,0.0701774,0.00396726,0.180533,0.00748821,0.354007,0.00717859,4.91501452148,2.2169832028,15.3996873047,3.9242435328,1090.30919375,33.0198303107,643.1602125,25.3606035516
0242p000,23.5680873831,0.237115319797,0.5559,0.00759816,0.000946752,0.00830058,0.000372407,0.017375,0.000569082,0.0508966,0.000970162,0.0298896,0.00333377,0.0347468,0.00520014,0.0569324,0.00409066,0.0582967,0.00707859,0.0574337,0.00636905,2.59826155762,1.61191239142,13.8701373047,3.72426332376,1077.61326562,32.8270203586,638.601925,25.2705742911
0287m016,29.3205056246,-0.884636819879,0.4223,0.0087676,0.00109288,0.0131056,0.000531681,0.0232964,0.00076599,0.0775727,0.00129397,0.0492031,0.00390114,0.0651956,0.00463052,0.0496609,0.00364349,0.0914773,0.0058773,0.140448,0.00575708,4.600758125,2.14493779047,14.4914378906,3.80676212688,1024.817225,32.012766594,594.69288125,24.3863257021
0317p015,31.7321267288,1.14073302163,0.4712,0.0193064,0.00163807,0.0265698,0.000744049,0.0498798,0.00108943,0.140604,0.0018933,0.129575,0.00564442,0.0693173,0.00538721,0.0776212,0.0037513,0.103825,0.00689423,0.126122,0.00644539,3.99281519287,1.99820299091,14.1576227051,3.76266165169,1055.92171875,32.4949491267,619.741484375,24.8946075361


In [9]:
t_opt.write("Fluxes.txt", format='ascii.tab')
t_opt.write("Fluxes.fits")